# 🔧 Signature Verification — Notebook #5 (Model Tuning & Stacking)
This notebook explores advanced model improvements:
- GridSearchCV tuning for XGBoost and MLP
- Weighted VotingClassifier
- StackingClassifier (meta-model)

We evaluate using Accuracy, F1, EER, and Confusion Matrix.

In [ ]:
!pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from xgboost import XGBClassifier
sns.set_style('whitegrid')

In [ ]:
DATA_DIR = '/content/drive/My Drive/ProjectLabDataset'
TRAIN_FILE = f'{DATA_DIR}/mcytTraining.txt'
TEST_FILE = f'{DATA_DIR}/mcytTesting.txt'
cols = ['ID', 'SigID', 'X', 'Y', 'P', 'al', 'az', 'signatureOrigin']
train_df = pd.read_csv(TRAIN_FILE, names=cols, skiprows=1)
test_df = pd.read_csv(TEST_FILE, names=cols, skiprows=1)
for df in [train_df, test_df]:
    df.columns = df.columns.str.strip()
    for col in ['X','Y','P','al','az']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
train_df['label'] = train_df['signatureOrigin'].map({'Genuine': 1, 'Forged': 0})
test_df['label'] = test_df['signatureOrigin'].map({'Genuine': 1, 'Forged': 0})
drop_cols = ['ID', 'SigID', 'signatureOrigin', 'al']
X_train = train_df.drop(columns=drop_cols + ['label'])
y_train = train_df['label']
X_test = test_df.drop(columns=drop_cols + ['label'])
y_test = test_df['label']

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
param_grid_xgb = {
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 150]
}
grid_xgb = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss'), param_grid_xgb, scoring='f1', cv=3)
grid_xgb.fit(X_train_scaled, y_train)
print("✅ Best XGBoost Params:", grid_xgb.best_params_)

In [ ]:
param_grid_mlp = {
    'hidden_layer_sizes': [(64,), (64, 32)],
    'alpha': [0.0001, 0.001],
    'solver': ['adam'],
    'max_iter': [500]
}
grid_mlp = GridSearchCV(MLPClassifier(random_state=42), param_grid_mlp, scoring='f1', cv=3)
grid_mlp.fit(X_train_scaled, y_train)
print("✅ Best MLP Params:", grid_mlp.best_params_)

In [ ]:
best_xgb = grid_xgb.best_estimator_
best_mlp = grid_mlp.best_estimator_

voting = VotingClassifier(estimators=[('xgb', best_xgb), ('mlp', best_mlp)], voting='soft', weights=[1, 2])
voting.fit(X_train_scaled, y_train)
y_pred_vote = voting.predict(X_test_scaled)
y_prob_vote = voting.predict_proba(X_test_scaled)[:, 1]
acc = accuracy_score(y_test, y_pred_vote)
f1 = f1_score(y_test, y_pred_vote)
print(f'✅ VotingClassifier — Accuracy: {acc:.3f}, F1: {f1:.3f}')

In [ ]:
def compute_eer(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    fnr = 1 - tpr
    idx = np.nanargmin(np.abs(fpr - fnr))
    eer = (fpr[idx] + fnr[idx]) / 2
    return eer, thresholds[idx]

eer, threshold = compute_eer(y_test, y_prob_vote)
print(f'🔍 VotingClassifier EER = {eer:.3f} at threshold = {threshold:.3f}')

stack = StackingClassifier(
    estimators=[('xgb', best_xgb), ('mlp', best_mlp)],
    final_estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss')
)
stack.fit(X_train_scaled, y_train)
y_pred_stack = stack.predict(X_test_scaled)
y_prob_stack = stack.predict_proba(X_test_scaled)[:, 1]
acc = accuracy_score(y_test, y_pred_stack)
f1 = f1_score(y_test, y_pred_stack)
print(f'✅ StackingClassifier — Accuracy: {acc:.3f}, F1: {f1:.3f}')
ConfusionMatrixDisplay.from_estimator(stack, X_test_scaled, y_test)
plt.title("Confusion Matrix — Stacking Classifier")
plt.show()